In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import pickle

import keras
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from vit_keras import vit

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.backend import clear_session


C:\Users\User\anaconda3\envs\GPU3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
data_path = r'C:\Users\User\Desktop\Master_Y1\Deep_learning\BrainTumorDetection-main\brain_tumor_dataset'
classes = ['no', 'yes']
def create_inception_v3():
    base_incep = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    x = base_incep.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(len(classes), activation='softmax')(x)
    inception = Model(inputs=base_incep.input, outputs=predictions)
    for layer in base_incep.layers:
        layer.trainable = False
    inception.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return inception

def create_resnet50():
    base_resn = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    x = base_resn.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(len(classes), activation='softmax')(x)
    resnet = Model(inputs=base_resn.input, outputs=predictions)
    for layer in base_resn.layers:
        layer.trainable = False
    resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return resnet

def create_vgg16():
    base_model = keras.applications.VGG16(weights='imagenet', include_top=False) 
    base_model.trainable = False
    inputs = keras.Input((224, 224, 3))
    x = base_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(2, activation='sigmoid')(x)
    VGG = keras.Model(inputs, outputs, name="VGG")
    VGG.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return VGG

def create_vit_b16():
    b16 = vit.vit_b16(image_size=(img_size, img_size), activation='sigmoid', include_top=True, pretrained_top=False, classes=2)
    b16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return b16

def create_vit_b32():
    b32 = vit.vit_b32(image_size=(img_size, img_size), activation='sigmoid', include_top=True, pretrained_top=False, classes=2)
    b32.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return b32

def create_vit_l16():
    l16 = vit.vit_l32(image_size=(224,224), activation='sigmoid', include_top=True, pretrained_top=False, classes=2)
    l16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return l16


In [3]:
def load_and_preprocess_data(img_size):
    data_path = r'C:\Users\User\Desktop\Master_Y1\Deep_learning\BrainTumorDetection-main\brain_tumor_dataset'
    classes = ['no', 'yes']

    X = []
    Y = []
    for c in classes:
        path = os.path.join(data_path, c)
        class_num = classes.index(c)
        for img in os.listdir(path):
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            img_arr = cv2.resize(img_arr, (img_size, img_size))
            X.append(img_arr)
            Y.append(class_num)

    X = np.array(X)
    Y = np.array(Y)
    Y = to_categorical(Y, num_classes=len(classes))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    return X, Y, kf

In [4]:
def cross_validate(create_model_fn, X, Y, kf, batch_size=8, epochs=10):
    val_accuracies = []
    val_losses = []

    best_history = None
    best_val_accuracy = -np.inf

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        Y_train, Y_val = Y[train_index], Y[val_index]

        clear_session()

        # Create a new instance of the model with reset weights
        model = create_model_fn()

        model_history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val))

        # Save the validation accuracy and validation loss for this fold
        current_val_accuracy = model_history.history['val_binary_accuracy'][-1]
        
        val_accuracies.append(current_val_accuracy)
        val_losses.append(model_history.history['val_loss'][-1])

        # Check if the current fold has the highest validation accuracy
        if current_val_accuracy > best_val_accuracy:
            best_val_accuracy = current_val_accuracy
            best_history = model_history
            
        del model
        del model_history
        del X_train
        del X_val
        del Y_train
        del Y_val
        gc.collect()

    # Calculate average validation accuracy and loss
    avg_val_accuracy = np.mean(val_accuracies)
    avg_val_loss = np.mean(val_losses)

    return avg_val_accuracy, avg_val_loss, best_history

In [5]:
img_size = 224
X, Y, kf = load_and_preprocess_data(img_size)

In [ ]:
avg_val_accuracy, avg_val_loss, inception_history = cross_validate(create_inception_v3, X, Y, kf, batch_size=8, epochs=5)

Epoch 1/5
26/26 [==============================] - 9s 95ms/step - loss: 24.4662 - binary_accuracy: 0.6584 - val_loss: 9.0180 - val_binary_accuracy: 0.6275
Epoch 2/5
26/26 [==============================] - 1s 37ms/step - loss: 6.6875 - binary_accuracy: 0.7525 - val_loss: 6.8528 - val_binary_accuracy: 0.7647
Epoch 3/5
26/26 [==============================] - 1s 37ms/step - loss: 4.2047 - binary_accuracy: 0.7921 - val_loss: 8.0756 - val_binary_accuracy: 0.5490
Epoch 4/5
26/26 [==============================] - 1s 37ms/step - loss: 2.7778 - binary_accuracy: 0.7624 - val_loss: 2.6543 - val_binary_accuracy: 0.8431
Epoch 5/5
26/26 [==============================] - 1s 36ms/step - loss: 1.9209 - binary_accuracy: 0.7921 - val_loss: 2.0547 - val_binary_accuracy: 0.8039
Epoch 1/5
26/26 [==============================] - 6s 94ms/step - loss: 12.2189 - binary_accuracy: 0.6931 - val_loss: 8.7054 - val_binary_accuracy: 0.7059
Epoch 2/5
26/26 [==============================] - 1s 36ms/step - loss: 4.

In [ ]:
file_path = "inception_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(inception_history.history, file_pi)

In [ ]:
avg_val_accuracy, avg_val_loss, resnet_history = cross_validate(create_resnet50, X, Y, kf, batch_size=8, epochs=50)

In [ ]:
file_path = "resnet_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(resnet_history.history, file_pi)

In [ ]:
avg_val_accuracy, avg_val_loss, vgg16_history = cross_validate(create_vgg16, X, Y, kf, batch_size=8, epochs=50)

In [ ]:
file_path = "vgg16_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(vgg16_history.history, file_pi)

In [ ]:
avg_val_accuracy, avg_val_loss, b16_history = cross_validate(create_vit_b16, X, Y, kf, batch_size=8, epochs=50)

In [ ]:
file_path = "b16_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(b16_history.history, file_pi)

In [ ]:
avg_val_accuracy, avg_val_loss, b32_history = cross_validate(create_vit_b32, X, Y, kf, batch_size=8, epochs=50)

In [ ]:
file_path = "b32_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(b32_history.history, file_pi)

In [ ]:
avg_val_accuracy, avg_val_loss, l16_history = cross_validate(create_vit_l16, X, Y, kf, batch_size=4, epochs=50)

In [ ]:
file_path = "l16_history.pickle"

with open(file_path, 'wb') as file_pi:
    pickle.dump(l16_history.history, file_pi)

In [ ]:
model_histories = {}
model_names = ['b32', 'b16', 'vgg16', 'resnet', 'inception', 'l16']

for model_name in model_names:
    file_path = f"{model_name}_history.pickle"
    with open(file_path, 'rb') as file_pi:
        model_histories[model_name] = pickle.load(file_pi)

In [ ]:
def evaluate_model(history):

    # Print the loss function
    print("Loss function:", history.history['loss'])

    # Plot the training and validation loss
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.show()

    # Print the training and validation accuracy
    train_acc = history.history['binary_accuracy']
    val_acc = history.history['val_binary_accuracy']
    print("Training Accuracy:", train_acc)
    print("Validation Accuracy:", val_acc)

#     # Plot the training and validation accuracy
#     plt.plot(train_acc, label='Training Accuracy')
#     plt.plot(val_acc, label='Validation Accuracy')
#     plt.legend()
#     plt.show()

#     # Calculate the confusion matrix for the validation set
#     Y_pred = model.predict(X_val)
#     Y_pred_classes = np.argmax(Y_pred, axis=1)
#     Y_val_classes = np.argmax(Y_val, axis=1)
#     cm = confusion_matrix(Y_val_classes, Y_pred_classes)
#     print("Confusion Matrix:")
#     print(cm)

#     # Plot the confusion matrix
#     sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
#     plt.show()

    # Create a table showing the training and validation loss and accuracy
    results_df = pd.DataFrame({'Loss': history.history['loss'], 'Accuracy': history.history['binary_accuracy'],
                               'Val_Loss': history.history['val_loss'], 'Val_Accuracy': history.history['val_binary_accuracy']})
    print(results_df)

#     # Get the indices and confidence scores of the misclassified images
#     misclassified_indices = np.where(Y_pred_classes != Y_val_classes)[0]
#     misclassified_confidence = np.max(Y_pred[misclassified_indices], axis=1)

#     # Plot the misclassified images with confidence scores
#     plt.figure(figsize=(20, 20))
#     for i, index in enumerate(misclassified_indices[:25]):
#         plt.subplot(5, 5, i+1)
#         plt.imshow(X_val[index])
#         plt.axis('off')
#         plt.title('Predicted: %s\nActual: %s\nConfidence: %.2f' % (Y_pred_classes[index], Y_val_classes[index], misclassified_confidence[i]))

    plt.show()

In [ ]:
evaluate_model(inception_history)